In [1]:
# pip install c3d

In [2]:
import c3d, csv, os

In [3]:
counter = 0
filepaths = []
for root, dirs, files in os.walk('C:\\Users\\sakhn\\Downloads\\gait_data\\data'):
    if len(files) > 1:
        for file in files:
            if 'ST' not in file:
                counter += 1    
                filepath = root + "\\" + file
                filepaths.append(filepath)
filepaths.sort()
print("Number of C3D files found:", counter)

Number of C3D files found: 1143


In [4]:
%%capture cap --no-stderr
# we only read the labels once from one file since they are the same in all files
file = "C:\\Users\\sakhn\\Downloads\\gait_data\\data\\2014001\\2014001_C1_01.c3d"
%run -i c3d-metadata $file

In [5]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

with open('markers.csv', mode='w') as participants:
    writer = csv.writer(participants)
    header1 = "Participant Speed Trial "
    header2 = "# # # "
    header3 = " "
    r = open('output.txt')
    for index, line in enumerate(r):
        if "POINT.LABELS[" in line:
            marker = line.split(' ')[2].split('\'')[1]
            header1 += marker + ' ' + marker + ' ' + marker + ' '
            header2 += "X Y Z" + ' '
    writer.writerow(header1.split(' '))
    writer.writerow(header2.split(' '))
    writer.writerow(header3)

    # frame data starts here
    count = 0
    for path in filepaths:
        handle = open(path, 'rb')
        reader = c3d.Reader(handle)
        ID = handle.name.split('\\')[7].split('_')[0]
        speed = handle.name.split('_')[2]
        trial = handle.name.split('_')[3].split('.')[0]
        counter = 0
        row = ID + " " + speed + " " + trial + " "
        for item in reader.read_frames():
            counter += 1
            for i in range(0, len(item[1])):
                # item[1] contains the 52 markers with 3 values each (X, Y, Z)
                row += str(item[1][i][0]) + ' ' + str(item[1][i][1]) + ' ' + str(item[1][i][2]) + ' '
            writer.writerow(row.split(' '))
            row = ID + " " + speed + " " + trial + " "
        count += 1
        # only write two participants to the CSV then exit ### comment out next 2 lines for full dataset
        #if count == 45:
         #   break